In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel

np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

In [2]:
dfx  = pd.read_excel('./COVID19.xlsx', sheet_name='data')
dfy  = pd.read_excel('./COVID19.xlsx', sheet_name='sample_info')

dfx = dfx[dfx.columns[1:]]
cols = ["p-%s" % c for c in dfx.columns]
dfx.columns = cols

X = dfx.values
Y = pd.get_dummies(dfy['class']).values.astype(float)

mp = AggMap(dfx, metric = 'correlation')
mp.fit(cluster_channels = 5, verbose = 0)
mp.save('./saved_model/aggmap.mp')

2020-10-25 14:11:28,516 - INFO - [bidd-aggmap] - Calculating distance ...
2020-10-25 14:11:28,517 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 88/88 [00:00<00:00, 758.49it/s]


2020-10-25 14:11:29,051 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-10-25 14:11:32,679 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-10-25 14:11:32,696 - INFO - [bidd-aggmap] - Finished


## 4-fold cross validation and repeat 5 times 

In [3]:
outer_fold = 4

run_all = []
for repeat_seed in [8, 16, 32, 64, 128]: #5 repeats random seeds
    
    outer = KFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(Y)), Y[:,0])

    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testx = X[test_idx]
        
        trainx = X[train_idx]
        trainY = Y[train_idx]

        trainX = mp.batch_transform(trainx, scale_method = 'standard')
        testX = mp.batch_transform(testx, scale_method = 'standard')
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        # fit the model
        clf = AggModel.MultiClassEstimator(epochs = 50,  batch_size = 4, verbose = 0)
        clf.fit(trainX, trainY)
        
        # save the model for explaination
        clf._model.save( './saved_model/seed_%s-%s.h5' % (repeat_seed, fold_num))
        
        # make prediction
        y_true = testY[:,0]
        y_pred = clf.predict(testX)[:,0]
        y_score = clf.predict_proba(testX)[:,0]

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        acc = (tp + tn) / sum([tn, fp, fn, tp])
        
        sensitivity = tp / sum([tp, fn])
        specificity = tn / sum([tn, fp])
        
        prc_auc = prc_auc_score(y_true, y_score)
        roc_auc = roc_auc_score(y_true, y_score)
        
        precision = tp / sum([tp, fp])
        recall =  tp / sum([tp, fn]) #equals to sensitivity
        
        
        res = {'fold': fold_num,
               'repeat_seed':repeat_seed,
               
               'accuracy':acc, 
               
               'prc_auc':prc_auc, 
               'roc_auc':roc_auc,

               'sensitivity': sensitivity, 
               'specificity': specificity,
               
               'precision':precision,
               'recall':recall,
               
               'F1': 2*precision*sensitivity/(precision+sensitivity)
              }
        
        run_all.append(res)

100%|##########| 91/91 [00:00<00:00, 1894.09it/s]



 input train and test X shape is (271, 10, 9, 5), (91, 10, 9, 5) 
MultiClassEstimator(batch_size=4, epochs=50, gpuid='0')


100%|##########| 91/91 [00:00<00:00, 2374.97it/s]



 input train and test X shape is (271, 10, 9, 5), (91, 10, 9, 5) 
MultiClassEstimator(batch_size=4, epochs=50, gpuid='0')


100%|##########| 90/90 [00:00<00:00, 1870.94it/s]


...


 input train and test X shape is (271, 10, 9, 5), (91, 10, 9, 5) 
MultiClassEstimator(batch_size=4, epochs=50, gpuid='0')


100%|##########| 90/90 [00:00<00:00, 2224.83it/s]



 input train and test X shape is (272, 10, 9, 5), (90, 10, 9, 5) 
MultiClassEstimator(batch_size=4, epochs=50, gpuid='0')


100%|##########| 90/90 [00:00<00:00, 2132.44it/s]



 input train and test X shape is (272, 10, 9, 5), (90, 10, 9, 5) 
MultiClassEstimator(batch_size=4, epochs=50, gpuid='0')


In [4]:
clf._model.count_params()

323138

In [5]:
df = pd.DataFrame(run_all)
df.to_excel('results.xlsx')
df

,fold,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall,F1
0,fold_00,8,0.945055,0.995818,0.994083,0.961538,0.923077,0.943396,0.961538,0.952381
1,fold_01,8,0.945055,0.990943,0.984520,0.947368,0.941176,0.964286,0.947368,0.955752
2,fold_02,8,0.911111,0.979841,0.979753,0.955556,0.866667,0.877551,0.955556,0.914894
3,fold_03,8,0.977778,0.995153,0.992557,0.982456,0.969697,0.982456,0.982456,0.982456
4,fold_00,16,0.978022,0.997892,0.996429,0.964286,1.000000,1.000000,0.964286,0.981818
5,fold_01,16,0.956044,0.992733,0.987616,0.964912,0.941176,0.964912,0.964912,0.964912
6,fold_02,16,0.944444,0.992718,0.991567,1.000000,0.880952,0.905660,1.000000,0.950495
7,fold_03,16,0.933333,0.982835,0.983000,0.960000,0.900000,0.923077,0.960000,0.941176
8,fold_00,32,0.890110,0.984197,0.971429,0.946429,0.800000,0.883333,0.946429,0.913793
9,fold_01,32,0.912088,0.985535,0.982353,0.980392,0.825000,0.877193,0.980392,0.925926


In [6]:
results = df.groupby(['repeat_seed']).apply(np.mean)
results

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall,F1
repeat_seed,,,,,,,,,
8,8.0,0.944750,0.990439,0.987728,0.961730,0.925154,0.941922,0.961730,0.951371
16,16.0,0.952961,0.991545,0.989653,0.972299,0.930532,0.948412,0.972299,0.959600
32,32.0,0.931105,0.991246,0.987070,0.967816,0.881250,0.921613,0.967816,0.943172
64,64.0,0.939316,0.988483,0.982286,0.938416,0.937040,0.957951,0.938416,0.946879
128,128.0,0.955800,0.993433,0.990033,0.966677,0.939098,0.958789,0.966677,0.962379


In [7]:
df.groupby('repeat_seed').apply(np.mean).mean().round(3)

repeat_seed    49.600
accuracy        0.945
prc_auc         0.991
roc_auc         0.987
sensitivity     0.961
specificity     0.923
precision       0.946
recall          0.961
F1              0.953
dtype: float64

In [8]:
df.groupby('repeat_seed').apply(np.std).mean().round(3)

repeat_seed    0.000
accuracy       0.020
prc_auc        0.005
roc_auc        0.007
sensitivity    0.025
specificity    0.046
precision      0.032
recall         0.025
F1             0.018
dtype: float64